# PDTB 句间关系分类 LoRA 模型合并
本脚本用于将 LoRA 微调后的权重与基础模型合并，并进行推理测试。

In [ ]:
import mindnlp
import mindspore
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig
from peft import LoraConfig, TaskType, get_peft_model, PeftModel

# 检查mindnlp版本
print("MindNLP版本:", mindnlp.__version__)

In [ ]:
# 实例化tokenizer
tokenizer = AutoTokenizer.from_pretrained('deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B', use_fast=False, trust_remote_code=True)
tokenizer

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    'deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B',
    ms_dtype=mindspore.bfloat16,
    device_map=0
)

In [ ]:
# 训练好的 checkpoint 路径
# 注意：这里需要修改为实际的checkpoint路径
lora_path = "./output/checkpoint-1380"

# 保存合并后模型的目录
merged_path = "./merged_qwen_lora"

In [ ]:
# 加载 LoRA 权重
model = PeftModel.from_pretrained(model, lora_path)

# 合并权重
model = model.merge_and_unload()

# 保存最终模型
model.save_pretrained(merged_path)
tokenizer.save_pretrained(merged_path)

print(f"LoRA 权重已合并完成，模型保存在: {merged_path}")

In [ ]:
# 推理测试
# 重新加载合并后的模型
merged_model = AutoModelForCausalLM.from_pretrained(
    merged_path,
    ms_dtype=mindspore.bfloat16,
    device_map=0
)

# 测试样本
test_samples = [
    "月亮又圆又亮，所以古人称之为玉盘。",
    "他的有没有什么不足之处？我觉得他可以就是加一些他自己的感受，因为他如果光只说那些一系列的动作，就感觉很空白，没有什么情感在里面。",
    "星汉是什么？银河。"
]

for sample in test_samples:
    print("\n输入:", sample)
    
    inputs = tokenizer.apply_chat_template([
        {"role": "system", "content": "你是一个中文篇章句间关系分析师"},
        {"role": "user", "content": sample}
    ], add_generation_prompt=True, tokenize=True, return_tensors="ms", return_dict=True)
    
    gen_kwargs = {"max_length": 200, "do_sample": True, "top_k": 1}
    with mindspore.no_grad():
        outputs = merged_model.generate(**inputs, **gen_kwargs)
        outputs = outputs[:, inputs['input_ids'].shape[1]:]
        result = tokenizer.decode(outputs[0], skip_special_tokens=True)
        print("输出:", result)